# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
import nltk
nltk.download(['punkt', 'wordnet', 'averaged_perceptron_tagger'])

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

In [10]:
# import libraries
import pandas as pd
import sqlite3
from sqlalchemy import create_engine
import re
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.multioutput import MultiOutputClassifier
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV
from sklearn.base import BaseEstimator, ClassifierMixin
import pickle

In [11]:
# load data from database
def load_data():
    engine = create_engine('sqlite:///DisasterResponse.db')
    df = pd.read_sql_table("disasterResponse", engine)
    X = df['message']
    Y = df.drop(['message', 'id', 'original', 'genre'], axis=1)
    
    return X, Y

### 2. Write a tokenization function to process your text data

In [12]:
#Tokenize function as explained in the course lesson
def tokenize(text):
    url_regex = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'
    detected_urls = re.findall(url_regex, text)
    for url in detected_urls:
        text = text.replace(url, "urlplaceholder")

    tokens = word_tokenize(text)
    lemmatizer = WordNetLemmatizer()

    clean_tokens = []
    for tok in tokens:
        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
        clean_tokens.append(clean_tok)

    return clean_tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [13]:
pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(RandomForestClassifier()))
]) 

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [14]:
X, y = load_data()
X_train, X_test, y_train, y_test = train_test_split(X, y)

In [15]:
model = pipeline
model.fit(X_train, y_train)

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [16]:
y_pred = model.predict(X_test)

In [17]:
for column in y_test.columns:
    print(classification_report(y_test[column], pd.DataFrame(y_pred, columns = y_test.columns)[column]))

             precision    recall  f1-score   support

          0       0.60      0.35      0.44      1528
          1       0.82      0.93      0.87      4981
          2       0.55      0.27      0.36        45

avg / total       0.77      0.79      0.77      6554

             precision    recall  f1-score   support

          0       0.89      0.98      0.93      5459
          1       0.81      0.38      0.52      1095

avg / total       0.88      0.88      0.86      6554

             precision    recall  f1-score   support

          0       0.99      1.00      1.00      6520
          1       0.00      0.00      0.00        34

avg / total       0.99      0.99      0.99      6554

             precision    recall  f1-score   support

          0       0.73      0.87      0.79      3905
          1       0.73      0.53      0.61      2649

avg / total       0.73      0.73      0.72      6554

             precision    recall  f1-score   support

          0       0.93      0.99 

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### 6. Improve your model
Use grid search to find better parameters. 

In [18]:
parameters = {
    
    'clf__estimator__n_estimators': [10],
    'clf__estimator__min_samples_split': [2, 4],
}

cv = GridSearchCV(pipeline, param_grid=parameters, verbose=4, n_jobs=-1)
cv.fit(X_train, y_train)

Fitting 3 folds for each of 2 candidates, totalling 6 fits
[CV] clf__estimator__min_samples_split=2, clf__estimator__n_estimators=10 
[CV]  clf__estimator__min_samples_split=2, clf__estimator__n_estimators=10, score=0.22093378089716204, total=  44.9s
[CV] clf__estimator__min_samples_split=2, clf__estimator__n_estimators=10 


[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed:   54.5s remaining:    0.0s


[CV]  clf__estimator__min_samples_split=2, clf__estimator__n_estimators=10, score=0.22078120231919438, total=  44.5s
[CV] clf__estimator__min_samples_split=2, clf__estimator__n_estimators=10 


[Parallel(n_jobs=-1)]: Done   2 out of   2 | elapsed:  1.8min remaining:    0.0s


[CV]  clf__estimator__min_samples_split=2, clf__estimator__n_estimators=10, score=0.20979554470552333, total=  44.7s
[CV] clf__estimator__min_samples_split=4, clf__estimator__n_estimators=10 


[Parallel(n_jobs=-1)]: Done   3 out of   3 | elapsed:  2.7min remaining:    0.0s


[CV]  clf__estimator__min_samples_split=4, clf__estimator__n_estimators=10, score=0.2171193164479707, total=  40.1s
[CV] clf__estimator__min_samples_split=4, clf__estimator__n_estimators=10 
[CV]  clf__estimator__min_samples_split=4, clf__estimator__n_estimators=10, score=0.1978944156240464, total=  39.9s
[CV] clf__estimator__min_samples_split=4, clf__estimator__n_estimators=10 
[CV]  clf__estimator__min_samples_split=4, clf__estimator__n_estimators=10, score=0.20216661580714068, total=  39.6s


[Parallel(n_jobs=-1)]: Done   6 out of   6 | elapsed:  5.2min finished


GridSearchCV(cv=None, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))]),
       fit_params=None, iid=True, n_jobs=-1,
       param_grid={'clf__estimator__n_estimators': [10], 'clf__estimator__min_samples_split': [2, 4]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=4)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [19]:
y_pred = model.predict(X_test)

In [20]:
for column in y_test.columns:
    print(classification_report(y_test[column], pd.DataFrame(y_pred, columns = y_test.columns)[column]))

             precision    recall  f1-score   support

          0       0.60      0.35      0.44      1528
          1       0.82      0.93      0.87      4981
          2       0.55      0.27      0.36        45

avg / total       0.77      0.79      0.77      6554

             precision    recall  f1-score   support

          0       0.89      0.98      0.93      5459
          1       0.81      0.38      0.52      1095

avg / total       0.88      0.88      0.86      6554

             precision    recall  f1-score   support

          0       0.99      1.00      1.00      6520
          1       0.00      0.00      0.00        34

avg / total       0.99      0.99      0.99      6554

             precision    recall  f1-score   support

          0       0.73      0.87      0.79      3905
          1       0.73      0.53      0.61      2649

avg / total       0.73      0.73      0.72      6554

             precision    recall  f1-score   support

          0       0.93      0.99 

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### 9. Export your model as a pickle file

In [22]:
with open('multioutclassifier.pickle', 'wb') as file:
    pickle.dump(model, file)

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.